In [88]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/python-utils-ak')
sys.path.append(r'C:\Users\arsen\Desktop\code\git\python-utils-ak')
from utils_ak.reflexion import load as _load

def _import_globals(module):
    for x in dir(module):
        if not x.startswith('_'):
            globals()[x] = getattr(module, x)
            
def load(module_obj, module_name=None, reload=False, import_globals=False):
    module = _load(module_obj, module_name, reload, import_globals=False)
    if import_globals:
        _import_globals(module)
    return module

load('src/time.py', import_globals=True, reload=True)
load('src/block.py', import_globals=True, reload=True)
load('src/color.py', import_globals=True, reload=True)
load('src/drawing.py', import_globals=True, reload=True)
load('src/interval.py', import_globals=True, reload=True)

<module 'src/interval.py' from 'src/interval.py'>

# Block

In [89]:
# ROWS = {'water_1': 6, 'water_2': 9, 'salt_1': 15, 'salt_2': 18, 'water_melting': 24, 'water_packing': 29, 'salt_melting': 33, 'salt_packing': 46}
ROWS = {'water_1': 1, 'water_2': 3, 'water_melting': 5, 'water_packing': 8}

def cast_row(obj):
    if isinstance(obj, int):
        return obj
    elif isinstance(obj, str):
        return ROWS[obj]
    else:
        raise Exception('Unknown object type')

In [90]:
def make_boiling(line):
    maker = BlockMaker(default_push_func=dummy_push)
    make = maker.make

    with make('boiling', block_num=12, line=line, size=27 + cast_t('04:00') + 31 + 20):
        total_size=27
        with make('pouring', y=cast_row(f'water_{line}'), size=total_size):
            with make(y=0):
                make('termizator', size=6, block_num=12)
                make('pouring_name', size=total_size - 6, name='чильеджина  3,6 альче  8500кг')
            with make(y=1):
                make('pouring_and_fermenting', size=8)
                make('soldification', size=7)
                make('cutting', size=7)
                make('pouring_off', size=3)
                make('extra', size=total_size - 8 - 8 - 7 - 3)

        make('drenator', size=cast_t('04:00'), visible=False)

        total_size = 31
        with make('melting', y=10, size=total_size):
            with make(y=0):
                make('melting_label', size=3)
                make('melting_name', size=total_size - 3, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
            with make(y=1):
                make('serving', size=6)
                make('melting_process', size=total_size - 6, speed=900)
            with make(y=2):
                make(size=6, visible=False)
                make('cooling1', size=5)
                make('cooling2', size=10)


        total_size = 20
        with make('packing', t=27 + cast_t('04:00') + 21, size=total_size, push_func=add_push):
            with make(y=0):
                make('packing_label', size=3)
                make('packing_name', size=total_size - 3, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
            with make(y=1):
                make('packing_brand', size=total_size, name='безлактозная/претто')
            with make(y=2):
                make('configuration', size=3, visible=False)
                make('configuration', size=1)
                make('configuration', size=5, visible=False)
                make('configuration', size=1)

    return maker.root.children[0]

In [92]:
def boiling_validator(parent, boiling):
    b1 = max(parent.children, key=lambda b: b.rel_props.get('t', 0))
    b2 = boiling
    
    b1.rel_props['props_mode'] = 'absolute'
    b2.rel_props['props_mode'] = 'absolute'
    
    validate_disjoint(b1['pouring'][0]['termizator'], b2['pouring'][0]['termizator'])
    
    b1.rel_props.pop('props_mode')
    b2.rel_props.pop('props_mode')

In [93]:
root = Block('root', t=cast_t('01:45'))
boiling_1 = make_boiling(1)
boiling_2 = make_boiling(2)
dummy_push(root, boiling_1);

In [94]:
dummy_push(root, boiling_2, validator=boiling_validator)
root

root (21, 21]
  boiling (0, 126]
    pouring (0, 27]
      block (0, 0]
        termizator (0, 6]
        pouring_name (6, 27]
      block (0, 0]
        pouring_and_fermenting (0, 8]
        soldification (8, 15]
        cutting (15, 22]
        pouring_off (22, 25]
        extra (25, 26]
    drenator (27, 75]
    melting (75, 106]
      block (0, 0]
        melting_label (0, 3]
        melting_name (3, 31]
      block (0, 0]
        serving (0, 6]
        melting_process (6, 31]
      block (0, 0]
        block (0, 6]
        cooling1 (6, 11]
        cooling2 (11, 21]
    packing (96, 116]
      block (0, 0]
        packing_label (0, 3]
        packing_name (3, 20]
      block (0, 0]
        packing_brand (0, 20]
      block (0, 0]
        configuration (0, 3]
        configuration (3, 4]
        configuration (4, 9]
        configuration (9, 10]
  boiling (127, 253]
    pouring (127, 154]
      block (127, 154]
        termizator (127, 133]
        pouring_name (6, 27]
      block (

In [95]:
style = {
    'pouring': {'beg_time': '01:00'},
    'termizator': {'text': '{block_num} налив'},
    'pouring_name': {'text': '{name}'},
    'pouring_and_fermenting': {'text': 'налив/внесение закваски', 'color': 'orange'},
    'soldification': {'text': 'схватка', 'color': 'yellow'},
    'cutting': {'text': 'резка/обсушка', 'color': '#92d050'},
    'pouring_off': {'text': 'слив', 'color': 'red'},
    'melting': {'beg_time': '07:00', 'y': cast_row('water_melting')},
    'melting_label': {'text': '{block_num}'},
    'melting_name': {'text': '{name}'},
    'serving': {'color': 'orange', 'text': 'подача и вымешивание'},
    'melting_process': {'color': 'orange', 'text': 'плавление {speed} кг/ч'},
    'cooling1': {'text': 'охлаж1 <{size} * 5> мин'},
    'cooling2': {'text': 'охлаж2 <{size} * 5> мин'},
    'packing': {'beg_time': '07:00', 'y': cast_row('water_packing')},
    'packing_label': {'text': '{block_num}'},
    'packing_name': {'text': '{name}'},
    'packing_brand': {'color': '#f2dcdb', 'text': '{name}'},
    'configuration': {'color': 'red'}
}

# update styles
for b in root.iter():
    if b.rel_props['class'] in style:
        b.rel_props.update(style[b.rel_props['class']])

In [96]:
work_book, sheet = init_sheet()
# work_book, sheet = init_template_sheet()
root.rel_props['index_width'] = 4
draw(sheet, root)
work_book.save('output.xlsx')

termizator 1 (14,20] {'t': 21, 'class': 'termizator', 'index_width': 4, 'block_num': 12, 'line': 1, 'size': 6, 'y': 1, 'beg_time': '01:00', 'text': '{block_num} налив'}
pouring_name 1 (20,41] {'t': 27, 'class': 'pouring_name', 'index_width': 4, 'block_num': 12, 'line': 1, 'size': 21, 'y': 1, 'beg_time': '01:00', 'name': 'чильеджина  3,6 альче  8500кг', 'text': '{name}'}
pouring_and_fermenting 2 (14,22] {'t': 21, 'class': 'pouring_and_fermenting', 'index_width': 4, 'block_num': 12, 'line': 1, 'size': 8, 'y': 2, 'beg_time': '01:00', 'text': 'налив/внесение закваски', 'color': 'orange'}
soldification 2 (22,29] {'t': 29, 'class': 'soldification', 'index_width': 4, 'block_num': 12, 'line': 1, 'size': 7, 'y': 2, 'beg_time': '01:00', 'text': 'схватка', 'color': 'yellow'}
cutting 2 (29,36] {'t': 36, 'class': 'cutting', 'index_width': 4, 'block_num': 12, 'line': 1, 'size': 7, 'y': 2, 'beg_time': '01:00', 'text': 'резка/обсушка', 'color': '#92d050'}
pouring_off 2 (36,39] {'t': 43, 'class': 'pour